In [127]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import table

from astroquery.mast import Catalogs
from tqdm import tqdm, trange

output_notebook()

Loading BokehJS ...

In [2]:
ttargets = pd.read_csv('../data/GI_S002.csv') # tess 2-min targets
ktargets = pd.read_csv('../data/K2Campaign19targets.csv') # all kepler target

kttargets = pd.read_csv('../data/wtv-K2Campaign19targets.csv') # kepler targets within TESS fov

In [3]:
c = SkyCoord(ra=ttargets.RA.values*u.degree, dec=ttargets.Dec.values*u.degree)  
catalog = SkyCoord(ra=ktargets.rafloat.values*u.degree, dec=ktargets.decfloat.values*u.degree)  
idx, d2d, d3d = c.match_to_catalog_sky(catalog)  

/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


In [4]:
# match things closer than 2 arcseconds
match = d2d.arcsec < 200

In [5]:
d2d.arcsecond.min()

523.8194642865161

In [6]:
p = figure()
p.scatter(ttargets.RA, ttargets.Dec, fill_color='black', size=10)
p.scatter(ktargets.rafloat, ktargets.decfloat, fill_color='red')
show(p)

# conclusion: no 2-min TESS targets are in the Kepler target list

# ok now lets do the Kepler targets that are in the TESS FFI images

In [10]:
c = SkyCoord(ra=kttargets.RA.values*u.degree, dec=kttargets.DEC.values*u.degree)  
catalog = SkyCoord(ra=ktargets.rafloat.values*u.degree, dec=ktargets.decfloat.values*u.degree)  
idx, d2d, d3d = c.match_to_catalog_sky(catalog)  

/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


In [11]:
matched = ktargets.iloc[idx]

In [12]:
sc = ['SC' in matched.loc[:, 'InvestigationIDs'].values[i] for i in range(matched.shape[0])]

matched[sc]

,EPIC,rafloat,decfloat,Kp,InvestigationIDs


# conclusion: no short cadence targets in TESS FFIs

In [247]:
coords = ['{:f} {:+f}'.format(matched.iloc[i].rafloat, matched.iloc[i].decfloat) for i in range(matched.shape[0])]

# jtable = Catalogs.query_region(coords[0], catalog="Tic", radius=0.02)
# for i,c in enumerate(tqdm(coords[1:]),start=1):
#     try:
#         catalogData = Catalogs.query_region(c, catalog="Tic", radius=0.02)
#         catalogData['EPIC'] = matched.iloc[i].EPIC
#         catalogData['rafloat'] = matched.iloc[i].rafloat
#         catalogData['decfloat'] = matched.iloc[i].decfloat
#         catalogData['InvestigationIDs'] = matched.iloc[i].InvestigationIDs
#         jtable = table.vstack([jtable, catalogData[0]])
#     except IndexError:
#         print('error')
#         continue
        

catalogData = Catalogs.query_region(coords[0], catalog="Tic", radius=0.02)
catalogData['EPIC'] = matched.iloc[0].EPIC
catalogData['rafloat'] = matched.iloc[0].rafloat
catalogData['decfloat'] = matched.iloc[0].decfloat
catalogData['InvestigationIDs'] = matched.iloc[0].InvestigationIDs

jtable = catalogData[0]
for i in trange(1, matched.shape[0]):
    try:
        c = '{:f} {:+f}'.format(matched.iloc[i].rafloat, matched.iloc[i].decfloat)
        catalogData = Catalogs.query_region(c, catalog="Tic", radius=0.02)
        catalogData['EPIC'] = matched.iloc[i].EPIC
        catalogData['rafloat'] = matched.iloc[i].rafloat
        catalogData['decfloat'] = matched.iloc[i].decfloat
        catalogData['InvestigationIDs'] = matched.iloc[i].InvestigationIDs
        if catalogData[0]['ID'].astype(int) > 10000000000:
            jtable = table.vstack([jtable, catalogData[1]])
        else:
            jtable = table.vstack([jtable, catalogData[0]])
    except IndexError:
        print('error')
        continue


    

# catalogData[['ID', 'ra', 'dec', 'pmRA', 'pmDEC',
#                        'eclong', 'eclat', 'Tmag', 'Vmag', 'Kmag', 'Teff',
#                        'rad', 'mass', 'd', ]]

100%|██████████| 170/170 [02:01<00:00,  1.40it/s]


In [248]:
# jtable.sort(keys='Tmag')
jtable[['ID', 'ra', 'dec', 'pmRA', 'pmDEC',
                       'eclong', 'eclat', 'Tmag', 'Vmag', 'Kmag', 'Teff',
                       'rad', 'mass', 'd', ]]

ID,ra,dec,pmRA,pmDEC,eclong,eclat,Tmag,Vmag,Kmag,Teff,rad,mass,d
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
5738930,348.025888,-12.086764,33.3159,4.20386,344.269694,-6.396994,14.985,16.304,13.239,4231.0,--,--,--
5738933,348.071899,-12.069173,0.893552,-1.86069,344.318285,-6.398427,15.317,15.715,14.043,5514.0,--,--,--
5725852,347.968908,-12.062366,31.45,11.21,344.227715,-6.35273,13.728,14.696,12.341,4870.0,0.818209,0.793837,457.56
5738934,348.029416,-12.051049,--,--,344.28696,-6.365482,16.638,--,15.087,4934.0,--,--,--
5725844,347.955744,-12.021335,9.55087,-7.3219,344.231954,-6.309933,12.825,13.793,11.777,5219.0,0.923422,0.8890419,465.288
5738942,348.087111,-11.994262,-49.27,-1.38,344.361577,-6.335325,16.62,--,14.03,--,--,--,--
5738973,348.161186,-11.988628,-1.44978,-28.4637,344.430879,-6.358521,12.44,13.152,11.529,5766.0,1.1282,1.03250551,489.412
5738970,348.15344,-11.982914,--,--,344.426116,-6.350295,16.68,--,14.687,4518.0,--,--,--
5738944,348.021353,-11.982817,-24.3069,-15.5453,344.306536,-6.299608,11.426,11.982,10.45,5831.0,1.15402,1.05066192,310.613


In [249]:
jp = jtable.to_pandas()

In [250]:
# jp.loc[:,['ID', 'ra', 'dec', 'Tmag', 'Vmag', 'Jmag', 'Teff']]

In [251]:
allids = np.array([])
q = ['{}'.format(matched.iloc[i].InvestigationIDs) for i in range(matched.shape[0])]
for i in q:
    allids = np.r_[allids,i.split('|')]

In [252]:
np.unique(allids, return_counts=True)

(array(['GO19003_LC', 'GO19011_LC', 'GO19016_LC', 'GO19026_LC',
        'GO19029_LC', 'GO19030_LC', 'GO19033_LC', 'GO19049_LC',
        'GO19051_LC', 'GO19053_LC', 'GO19059_LC', 'GO19063_LC',
        'GO19065_LC', 'GO19067_LC', 'GO19068_LC'], dtype='<U32'),
 array([  1,  11,   1,   2,   1,   1,   1,  33,   1,   3,  36,  19,  17,
        134,  90]))

In [253]:
# there are some dodge matches that we want to fix

jp.ID[jp.ID.astype(int) > 10000000000]


Series([], Name: ID, dtype: object)

In [254]:
jp.to_csv('tesskepler_matched.csv')

In [255]:
coords = ['{:f} {:+f}'.format(matched.iloc[i].rafloat, matched.iloc[i].decfloat) for i in range(matched.shape[0])]

for i in range(150,170):
    print(Catalogs.query_region(coords[i], catalog="Tic", radius=0.001)[['ID', 'ra', 'dec',
                           'Tmag', 'Vmag', 'Kmag', 'Teff',
                           'dstArcSec']])
    print()


   ID      ra      dec     Tmag   Vmag  Kmag   Teff       dstArcSec     
------- -------- -------- ------ ----- ------ ------ -------------------
4611200 349.0654 -11.2858 17.088 19.37 14.297 3306.0 0.12196353266946258

     ID         ra        dec      Tmag  Vmag  Kmag   Teff       dstArcSec     
----------- ---------- ---------- ------ ---- ------ ------ -------------------
    4619642 349.204955 -11.282415 16.658   -- 14.287 3950.0 0.01766682333990753
10000201587 349.205048 -11.282425 16.561   -- 13.626     --  0.3126745051544306

   ID      ra      dec     Tmag   Vmag   Kmag   Teff       dstArcSec     
------- -------- -------- ------ ------ ------ ------ -------------------
4610899 349.0392 -11.2651 13.981 15.866 11.828 3769.0 0.31352278736998235

   ID       ra        dec      Tmag   Vmag   Kmag   Teff       dstArcSec     
------- ---------- ---------- ------ ------ ------ ------ -------------------
4619635 349.180483 -11.248313 15.218 16.683 13.224 3987.0 0.06403992657472748

 

 ID  ra dec Tmag Vmag Kmag Teff dstArcSec
--- --- --- ---- ---- ---- ---- ---------

 ID  ra dec Tmag Vmag Kmag Teff dstArcSec
--- --- --- ---- ---- ---- ---- ---------

     ID         ra        dec      Tmag  Vmag  Kmag   Teff       dstArcSec     
----------- ---------- ---------- ------ ---- ------ ------ -------------------
10000201313 348.278961 -11.809827 16.403   -- 13.492     -- 0.30188164759620484
    4579843 348.279166 -11.809883 18.006   -- 14.858 7426.0  0.6644450287152504

     ID         ra        dec      Tmag  Vmag  Kmag   Teff      dstArcSec     
----------- ---------- ---------- ------ ---- ------ ------ ------------------
10000201200 347.835388 -11.808946 15.192   --  13.53     -- 0.8999450705348463
    4520629 347.835094 -11.809527 15.668   -- 14.868 6695.0 1.5288121052355814



In [256]:
jp

,ID,ra,dec,pmRA,pmDEC,Tmag,objType,typeSrc,version,HIP,...,numcont,contratio,disposition,duplicate_id,priority,dstArcSec,EPIC,rafloat,decfloat,InvestigationIDs
0,5738930,348.025888,-12.086764,33.315900,4.203860,14.985,STAR,2mass,20180610,NaN,...,NaN,NaN,NaN,NaN,NaN,0.138963,245922798.0,348.02588,-12.086726,GO19067_LC|GO19068_LC
1,5738933,348.071899,-12.069173,0.893552,-1.860690,15.317,STAR,2mass,20180610,NaN,...,NaN,NaN,NaN,NaN,NaN,0.116868,245923096.0,348.07187,-12.069157,GO19067_LC|GO19068_LC
2,5725852,347.968908,-12.062366,31.450000,11.210000,13.728,STAR,2mass,20180610,NaN,...,10.0,1.618850e-08,NaN,NaN,NaN,0.107437,245923200.0,347.96890,-12.062337,GO19067_LC|GO19068_LC
3,5738934,348.029416,-12.051049,NaN,NaN,16.638,STAR,2mass,20180610,NaN,...,NaN,NaN,NaN,NaN,NaN,0.014124,245923374.0,348.02942,-12.051049,GO19059_LC
4,5725844,347.955744,-12.021335,9.550870,-7.321900,12.825,STAR,2mass,20180610,NaN,...,12.0,2.585064e-04,NaN,NaN,0.000332,0.105811,245923832.0,347.95575,-12.021306,GO19049_LC|GO19067_LC
5,5738942,348.087111,-11.994262,-49.270000,-1.380000,16.620,STAR,2mass,20180610,NaN,...,NaN,NaN,NaN,NaN,NaN,0.066969,245924338.0,348.08713,-11.994261,GO19063_LC
6,5738973,348.161186,-11.988628,-1.449780,-28.463700,12.440,STAR,2mass,20180610,NaN,...,15.0,NaN,NaN,NaN,NaN,0.334113,245924453.0,348.16110,-11.988667,GO19049_LC|GO19065_LC|GO19067_LC
7,5738970,348.153440,-11.982914,NaN,NaN,16.680,STAR,2mass,20180610,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000720,245924572.0,348.15344,-11.982914,GO19059_LC
8,5738944,348.021353,-11.982817,-24.306900,-15.545300,11.426,STAR,2mass,20180610,NaN,...,22.0,2.735549e-03,NaN,NaN,0.000518,0.190189,245924577.0,348.02133,-11.982769,GO19049_LC|GO19065_LC|GO19067_LC
9,5725841,347.953354,-11.980719,NaN,NaN,16.411,STAR,2mass,20180610,NaN,...,NaN,NaN,NaN,NaN,NaN,0.056350,245924621.0,347.95337,-11.980719,GO19059_LC
